# Setup

In [1]:
from splink import (
    splink_datasets,
    Linker,
    SettingsCreator,
    block_on,
    DuckDBAPI
)
import splink.comparison_library as cl

# Load Data

In [2]:
df = splink_datasets.fake_1000

# Setup Linker

In [8]:
settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        cl.LevenshteinAtThresholds("dob", 1),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "city"),
        block_on("surname"),

    ],
    retain_intermediate_calculation_columns=True,
)

In [6]:
linker = Linker(df, settings, db_api=DuckDBAPI())

# Estimation

## Probability Two Random Records Match

In [9]:
deterministic_rules = [
    block_on("first_name", "dob"),
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email")
]

linker.training.estimate_probability_two_random_records_match(
    deterministic_rules,
    recall=0.7
)

Probability two random records match is estimated to be  0.00298.
This means that amongst all possible pairwise record comparisons, one in 335.56 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 1,488.57 matching pairs


## u-Probability

In [11]:
linker.training.estimate_u_using_random_sampling(
    max_pairs=1e9
)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


## m-Probability

### Direct Estimation

In [13]:
# If this column existed, we could do this:
# linker.training.estimate_m_from_label_column(
#     "social_security_number"
# )

### Expectation Maximisation

#### First Pass

In [14]:
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(
        block_on(
            "first_name",
            "surname"
        )
    )
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.521 in the m_probability of dob, level `Exact match on dob`
Iteration 2: Largest change in params was 0.0516 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0183 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.00744 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00349 in probability_two_random_records_match
Iteration 6: Larg

#### Second Pass

In [15]:
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("dob")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.407 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.0929 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0548 in the m_probability of first_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.0186 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00758 in probability_two_random_records_match
Iteration 6: Largest change in p

# Visualise Params

## Match Weights

In [16]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

## m and u Parameters

In [17]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

## Compare Passes

In [18]:
linker.visualisations.parameter_estimate_comparisons_chart()

alt.Chart(...)

# Save

In [19]:
settings = linker.misc.save_model_to_json(
    "02_saved_model.json",
    overwrite=True
)

# Detecting Unlinkable Records

In [20]:
linker.evaluation.unlinkables_chart()

alt.LayerChart(...)